In [ ]:
-- daily orders > delays > voucher generation by country for funnel visualization, orders by country, map visuals
WITH daily_delays_vouchers AS
(
    SELECT 
        CAST(DATE_TRUNC(event_ts, day) AS date) AS date,
        country,
        COUNT(event_id) AS total_delays, 
        SUM(CASE WHEN action_type = 'no_action' THEN 1 ELSE 0 END) AS num_no_action,
        SUM(CASE WHEN action_type = 'absolute_voucher' THEN 1 ELSE 0 END) AS num_abs_voucher
    FROM `bq-sql-practice.foodpanda_interview.events`
    GROUP BY 1, 2
    ORDER BY 1
),
daily_total_orders AS
(
    SELECT 
        date,
        country,
        SUM(orders) AS total_orders
    FROM `bq-sql-practice.foodpanda_interview.orders`
    GROUP BY 1, 2
    ORDER BY 1
)
SELECT
    daily_total_orders.date,
    daily_total_orders.country,
    total_orders,
    total_delays,
    num_no_action,
    num_abs_voucher
FROM daily_total_orders
JOIN daily_delays_vouchers
    ON daily_total_orders.date = daily_delays_vouchers.date
    AND daily_total_orders.country = daily_delays_vouchers.country
ORDER BY 1

In [ ]:
-- Timeseries of daily orders/delays/voucher/no actions to visualization line plot
WITH daily_delays_vouchers AS
(
    SELECT 
        CAST(DATE_TRUNC(event_ts, day) AS date) AS date,
        country,
        COUNT(event_id) AS total_delays, 
        SUM(CASE WHEN action_type = 'no_action' THEN 1 ELSE 0 END) AS num_no_action,
        SUM(CASE WHEN action_type = 'absolute_voucher' THEN 1 ELSE 0 END) AS num_abs_voucher
    FROM `bq-sql-practice.foodpanda_interview.events`
    GROUP BY 1, 2
    ORDER BY 1
),
daily_total_orders AS
(
    SELECT 
        date,
        country,
        SUM(orders) AS total_orders
    FROM `bq-sql-practice.foodpanda_interview.orders`
    GROUP BY 1, 2
    ORDER BY 1
)
SELECT
    daily_total_orders.date,
    daily_total_orders.country,
    total_orders,
    total_delays,
    num_no_action,
    num_abs_voucher
FROM daily_total_orders
JOIN daily_delays_vouchers
    ON daily_total_orders.date = daily_delays_vouchers.date
    AND daily_total_orders.country = daily_delays_vouchers.country
ORDER BY 1

In [ ]:
-- Post voucher/no action survey ratings by day and customer_id for customer survey ratings visualization
WITH post_voucher_survey AS
(SELECT
    events.customer_id AS customer_id,
    event_ts,
    country,
    survey_response_ts,
    action_type,
    survey_rating
FROM `bq-sql-practice.foodpanda_interview.events` AS events
LEFT JOIN `bq-sql-practice.foodpanda_interview.customer_surveys` AS surveys
    ON events.customer_id = surveys.customer_id
WHERE survey_response_ts > event_ts)
SELECT
    CAST(DATE_TRUNC(event_ts, day) AS date) AS date,
    action_type,
    country,
    COUNT(customer_id) AS num_customers,
    AVG(survey_rating) AS avg_rating
FROM post_voucher_survey
GROUP BY 1, 2, 3
ORDER BY 1, 2, 3

In [ ]:
-- CLV and Survey ratings by day and country
WITH survey_after_event_voucher AS
(
    SELECT
        events.customer_id AS customer_id,
        event_ts,
        country,
        survey_response_ts,
        action_type,
        survey_rating,
        customer_value_index
    FROM `bq-sql-practice.foodpanda_interview.events` AS events
    LEFT JOIN `bq-sql-practice.foodpanda_interview.customer_surveys` AS surveys
        ON events.customer_id = surveys.customer_id
    WHERE survey_response_ts > event_ts
)
SELECT 
    customer_id,
    action_type,
    CAST(DATE_TRUNC(survey_response_ts, day) AS date) AS survey_response_date,
    CAST(DATE_TRUNC(event_ts, day) AS date) AS event_date,
    DATE_DIFF(CAST(DATE_TRUNC(survey_response_ts, day) AS date), CAST(DATE_TRUNC(event_ts, day) AS date), day) AS days_diff,
    country,
    survey_rating,
    customer_value_index
FROM survey_after_event_voucher